# 股票数据定向爬虫

** 功能描述 **

- 目标：获取上交所和深交所所有股票的名称和交易信息
- 输出：保存到文件中
- 技术路线：requests‐bs4‐re

** 程序结构设计 **

- 步骤1：从东方财富网获取股票列表
- 步骤2：根据股票列表逐个到百度股票获取个股信息
- 步骤3：将结果存储到文件

** 代码编写 **

In [1]:
import requests
from bs4 import BeautifulSoup
import traceback
import re

In [2]:
def getHTMLText(url, code = 'utf-8'):
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = code
        return r.text
    except:
        return ""

In [3]:
def getStockList(lst, stockURL):
    html = getHTMLText(stockURL, "GB2312")
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find_all('a')
    for i in a:
        try:
            href = i.attrs['href']
            lst.append(re.findall(r'[s][hz]\d{6}', href)[0])
        except:
            continue

In [4]:
def getStockInfo(lst, stockURL, fpath):
    count = 0
    for stock in lst:
        url = stockURL + stock + ".html"
        html = getHTMLText(url)
        try:
            if html == "":
                continue
            infoDict = {}
            soup = BeautifulSoup(html, 'html.parser')
            stockInfo = soup.find('div', attrs = {'class':'stock-bets'})
            
            name = stockInfo.find_all(attrs = {'class':'stock-name'})[0]
            infoDict.update({'股票名称':name.text.split()[0]})
            
            keyList = stockInfo.find_all('dt')
            valueList = stockInfo.find_all('dd')
            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val
                
            with oprn(fpath, 'a', encoding = 'utf-8') as f:
                f.write(str(infoDict) + '\n')
                count = count + 1
                print('\r当前进度:{:.2f}%'.format(count*100/len(lst)), end = "")
        except:
            count = count + 1
            print('\r当前进度:{:.2f}%'.format(count*100/len(lst)), end = "")
            continue

In [5]:
def main():
    stock_list_url = 'http://quote.eastmoney.com/stocklist.html'
    stock_info_url = 'https://gupiao.baidu.com/stock/'
    output_file = 'C://Users//FHT//Documents//GitHub//MyStudy//Computer//Python//Code//Python网络爬虫与信息提取//BaiduStockInfo.txt'
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)

In [6]:
main()

当前进度:99.95%